##  Import the necessary modules.<br>
Note: should you not have any of these packages installed, please use the following commands:<br>
pip install opencv-python<br>
pip install numpy<br>
pip install matplotlib<br>
pip install scikit-image<br>
pip install basicsudoku<br>

In [4]:
import os.path
import sys
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from skimage import io,morphology,measure
from skimage.metrics import structural_similarity as ssim
import basicsudoku
import operator

In order to solve a sudoku puzzle in an image, please set the variable image to the name of the image containing the puzzle. To test the program, please use one of the images in the images folder. The naming convention is "image" and the number of the image (between 1 -50)

In [5]:
image="image23"
#Import the image
org_puzzle=cv.imread("../Images/"+image+".jpg",0)
#Resize the image to a more manageable size
puzzle=cv.resize(org_puzzle,(990,990),interpolation=cv.INTER_AREA)
# Threshold the image such pixels with an intensity less than 70 gets mapped to 
#0 and pixels with an intensity greater than 70 gests mapped to 255
thr=cv.threshold(puzzle,70,255,cv.THRESH_BINARY_INV)[1]#

Define a cross structuring element (with size 3x3) which we use to perform an opening operation followed by a dilation operation. This is for noise removal purposes.


In [6]:
cross=np.array([
    [0,1,0],
    [1,1,1],
    [0,1,0]
]).astype(np.uint8)

Perform noise removal


In [7]:
result=cv.erode(thr,cross,borderType=cv.BORDER_CONSTANT)
result=cv.dilate(result,cross,borderType=cv.BORDER_CONSTANT)
result=cv.dilate(result,cross,borderType=cv.BORDER_CONSTANT)
thr=result

Define a function to compute the Euclidean Distance between 2 points.

In [8]:
def Euclidean_distance(point1, point2):
    return np.sqrt(((point2[0]-point1[0]) ** 2) + ((point2[1]-point1[1]) ** 2))

Process the contours of the image and locate the corners of the grid in the old region and define our new (square) region. This is the pre-processing required for performing the perspective transformation in the next step.

In [9]:
contours, h = cv.findContours(thr.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv.contourArea, reverse=True)
large_square = contours[0]


bottom_right, _ = max(enumerate([point[0][0] + point[0][1] for point in large_square]), key=operator.itemgetter(1))
top_left, _ = min(enumerate([point[0][0] + point[0][1] for point in large_square]), key=operator.itemgetter(1))
bottom_left, _ = min(enumerate([point[0][0] - point[0][1] for point in large_square]), key=operator.itemgetter(1))
top_right, _ = max(enumerate([point[0][0] - point[0][1] for point in large_square]), key=operator.itemgetter(1))

the_ROI=[large_square[top_left][0], large_square[top_right][0], large_square[bottom_right][0], large_square[bottom_left][0]]

top_left, top_right, bottom_right, bottom_left = the_ROI[0], the_ROI[1], the_ROI[2], the_ROI[3]
old_square = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32') 
long_side = max([ Euclidean_distance(bottom_right, top_right), 
Euclidean_distance(top_left, bottom_left),
 Euclidean_distance(bottom_right, bottom_left), 
 Euclidean_distance(top_left, top_right) ])
new_square = np.array([[0, 0], [long_side - 1, 0], [long_side - 1, long_side - 1], [0, long_side - 1]], dtype='float32')

Perform the perspective transformation and determine the size of the cells in the grid.

In [10]:
persp_trans = cv.getPerspectiveTransform(old_square, new_square)
final_answer=cv.warpPerspective(thr, persp_trans, (int(long_side), int(long_side)))
partial_height,partial_width=(int(final_answer.shape[0]/9),int(final_answer.shape[1]/9))

Store our templates in a list for easy retrieval when performing template matching.

In [11]:
templates=[]
for i in range(9):
    templates.append(cv.imread("../data/MeanNumbers/mean_number"+str(i+1)+".png",0))

Define an empty sudoku board

In [12]:
board = basicsudoku.SudokuBoard() 
board.strict=False
sim_values=np.zeros(9)

Process each cell where we extract a cell and determine whether it is empty or not. We determine this by checking if the average intensity of the cell is greater than 20. When we encounter a non-empty cell, we perform template matching with each of the templates and record the max_val(max value) acheived which is returned my cv.minmaxLoc(). This is used as a metrix for determining how well a template corresponds to the contains of the current cell. The template that yields the greatest max_val values determines the value that is entered in the digital grid. Once we process each cell, we have our final digital representation of the sudoku puzzle in the input image.

In [13]:
for i in range(9):
    for j in range(9):
        cell=final_answer[i*partial_height+12:(i+1)*partial_height-7,j*partial_width+12:(j+1)*partial_width-7]
        if(np.mean(cell)>20):
            for k in range(9):
                template = templates[k]
                res = cv.matchTemplate(cell,template,cv.TM_CCOEFF_NORMED)
                min_val, max_val, min_loc, max_loc=cv.minMaxLoc(res)
                sim_values[k]=max_val
            index=np.where(sim_values==np.max(sim_values))[0]
            if(index.size==1):
                board[j,i]=int(index+1)

Display the results and verify that the results produced are correct. The digital representation is printed below and an external window will open displaying the input image.

In [15]:
print(board)
cv.namedWindow('Input Image',cv.WINDOW_NORMAL)
cv.resizeWindow('Input Image', 600,600)
cv.imshow("Input Image",org_puzzle)
cv.waitKey(0)
cv.destroyAllWindows()

. 8 4 | . . 2 | . . .
. . . | . . 4 | . 2 .
. . 7 | 5 3 . | . . .
------+-------+------
. . . | . 5 . | . 3 2
. 6 . | 4 . 8 | . 9 .
8 2 . | . 1 . | . . .
------+-------+------
. . . | . 9 3 | 7 . .
. 4 . | 1 . . | . . .
. . . | 7 . . | 3 5 .
